![imagenes](logo.png)

# Clasificación binaria en redes neuronales

Cuando hablamos de clasificación, muchas personas piensan inmediatamente en separar datos en dos grupos: sí o no, verdadero o falso, positivo o negativo. En efecto, la clasificación binaria consiste en asignar cada observación a una de dos clases posibles.

Sin embargo, en deep learning, la clasificación no se reduce a “trazar una frontera”. Lo que realmente aprende el modelo es una función de decisión compleja que separa regiones del espacio de características de acuerdo con patrones estadísticos aprendidos a partir de los datos.

La diferencia fundamental con la regresión no está en la arquitectura interna de la red, sino en el tipo de variable objetivo y en la interpretación probabilística de la salida.

## ¿Qué significa clasificar en una red neuronal?

En un problema de clasificación binaria, el objetivo no es predecir un valor continuo, sino estimar la probabilidad de que una observación pertenezca a una de dos clases.

Formalmente, la red aprende una función $$f(\boldsymbol{x})=P(y=1|\boldsymbol{x})$$

donde el resultado está acotado entre 0 y 1.

Esto se logra utilizando una función de activación **sigmoide en la capa de salida**, que transforma cualquier valor real en una probabilidad interpretable.

Aunque el resultado final es un número entre 0 y 1, la complejidad real del modelo ocurre en las capas ocultas: allí se aprenden combinaciones no lineales de las variables de entrada que permiten separar regiones del espacio de características de forma altamente no lineal.

## Capa de salida en clasificación binaria

En clasificación binaria, la arquitectura de salida es prácticamente estándar:

- Una sola neurona

- Función de activación sigmoide

La salida puede interpretarse como:

- $\hat{y}\sim 1$ significa alta probabilidad de pertenecer a la clase positiva
- $\hat{y}\sim 0$ significa alta probabilidad de pertenecer a la clase negativa

El **umbral de decisión** puede ajustarse según el problema, por ejemplo, cuando el costo de falsos positivos y falsos negativos no es simétrico.

## Función de pérdida en clasificación binaria

En clasificación, la métrica central durante el entrenamiento no es una distancia, sino una función de verosimilitud.

La función estándar es la entropía cruzada binaria (binary cross-entropy): $$\mathcal{L}=-(y\log(\hat{y})+(1-y)\log(1-\hat{y}))$$

Esta función penaliza fuertemente las predicciones seguras pero incorrectas, y está directamente alineada con la interpretación probabilística del modelo.

Durante el entrenamiento, minimizar esta pérdida equivale a maximizar la verosimilitud de los datos observados bajo el modelo.

## Regularización en clasificación binaria

Al igual que en regresión, una red neuronal puede sobreajustar fácilmente los datos de entrenamiento, aprendiendo patrones espurios o ruido.

Las estrategias de regularización más comunes son:

- **Regularización L2:** Penaliza pesos grandes, favoreciendo fronteras de decisión más suaves y estables.

- **Dropout:** Desactiva aleatoriamente neuronas durante el entrenamiento, obligando a la red a no depender de rutas específicas de activación.

- **Early stopping:** Monitorea el desempeño en un conjunto de validación y detiene el entrenamiento cuando la mejora se estanca, evitando que la red memorice el conjunto de entrenamiento.

En clasificación, estas técnicas son especialmente importantes porque pequeñas variaciones en la frontera de decisión pueden cambiar la clase asignada.



## Métricas de evaluación en clasificación binaria

Aunque la función de pérdida guía el entrenamiento, la evaluación del modelo se basa en métricas interpretables:

**Accuracy:** proporción de predicciones correctas.

**Precision:** qué fracción de las predicciones positivas son correctas.

**Recall:** qué fracción de los positivos reales fue detectada.

**F1-score:** balance entre precisión y exhaustividad.

La elección de la métrica depende del problema: no es lo mismo detectar fraude que filtrar spam o diagnosticar una enfermedad.

## Pseudocódigo para diseñar una red para regresión binaria

```python
# ============================================================
# Diseñador de arquitectura para clasificación binaria
# (red neuronal con regularización y control de complejidad)
# ============================================================

k     ← 10            # máximo de parámetros por dato
c1    ← 2.0           # factor de expansión inicial
r     ← 0.5           # tasa de reducción entre capas
n_min ← 8             # tamaño mínimo de capa oculta
L_max ← 4             # máximo de capas ocultas

ALGORITHM DiseñarRedClasificacion(d, n0)

    # 0. Límite adaptativo por tamaño de muestra
    n_max ← MIN(1024, MAX(64, floor(0.25 * d)))

    # 1. Verificación mínima de viabilidad
    IF d < 2 * n0 THEN
        RETURN "Dataset demasiado pequeño para clasificación confiable"
    END IF

    # 2. Presupuesto total de parámetros
    P_max ← floor(k * d)

    # 3. Primera capa oculta
    n1_cap ← floor(P_max / (n0 + 1))
    n1 ← MIN(floor(c1 * n0), n1_cap, n_max)

    IF n1 < n_min THEN
        RETURN "Modelo demasiado pequeño para capturar patrones"
    END IF

    capas ← [n1]

    # 4. Construcción progresiva de capas
    WHILE length(capas) < L_max DO

        n_prev ← last(capas)
        n_new ← floor(r * n_prev)

        IF n_new < n_min THEN
            BREAK
        END IF

        append n_new to capas

    END WHILE

    # 5. Cálculo de parámetros totales
    P ← (n0 + 1) * capas[1]
    FOR i FROM 2 TO length(capas) DO
        P ← P + (capas[i-1] + 1) * capas[i]
    END FOR
    P ← P + (last(capas) + 1) * 1   # salida binaria

    # 6. Regularización adaptativa
    rho ← P / P_max

    IF rho >= 0.8 THEN
        l2 ← 3e-4
        dropout ← 0.4
    ELSE IF rho >= 0.4 THEN
        l2 ← 1e-4
        dropout ← 0.25
    ELSE
        l2 ← 5e-5
        dropout ← 0.15
    END IF

    # 7. Early stopping
    patience ← IF d < 2000 THEN 20 ELSE 10
    max_epochs ← IF d < 2000 THEN 300 ELSE 150

    RETURN (capas, l2, dropout, patience, max_epochs)

END ALGORITHM


```